In [ ]:
# Import libraries
import os
import sys
import numpy as np
import pandas as pd
import shutil
import matplotlib.pyplot as plt
import re
from skimage import io
import tifffile
from scipy.ndimage import median_filter

# Set Paths
#codedir = '/home/groups/graylab_share/OMERO.rdsStore/engje/Data/cmIF'
codedir = os.getcwd()
rootdir = f'{codedir}'
#tiffdir = f'{rootdir}/RawImages'
qcdir = f'{rootdir}/QC'
#regdir = f'{rootdir}/RegisteredImages'
subdir = f'{rootdir}/SubtractedRegisteredImages'
segdir = f'{rootdir}/Segmentation'
#czidir = f'{rootdir}/czis'

In [ ]:
# Start Preprocessing
os.chdir('/home/groups/graylab_share/data/engje/Data/')
from mplex_image import preprocess, mpimage, cmif
os.chdir(codedir)
preprocess.cmif_mkdir([qcdir,segdir,subdir])

# List slides to be processed
ls_sample = [
 ]


In [ ]:
import scipy
from skimage.filters import unsharp_mask
import skimage
from skimage.restoration import (denoise_tv_chambolle, denoise_bilateral,
                                 denoise_wavelet, estimate_sigma)
from skimage import color, morphology
from skimage.transform import rescale

# image renaming

In [ ]:
os.chdir(f'{codedir}/Images')

In [ ]:
df_s = pd.read_csv(f'{codedir}/proteins_by_frame.csv',index_col=0)

In [ ]:
df_s.Biomarker.unique()

In [ ]:
'''DONE
se_cyto = pd.Series(['Beta catenin','Vimentin','CD31','CD45'])
for s_file in os.listdir():
    if s_file.find('.tiff') > -1:
        print(s_file)
        s_slide = s_file.split('Point')[1].split('.')[0]
        s_scene = '1'
        imgs_cyto = []
        with tifffile.TiffFile(s_file) as tif:
            array = tif.asarray()
            #omexml_string = tif.ome_metadata
            for idx in range(array.shape[0]):
                img = array[idx]
                img = np.clip(img, a_min=0, a_max=65535).astype('uint16')
                s_marker = df_s.loc[idx,'Biomarker']
                if s_marker =='dsDNA':
                    dapi_scale = skimage.exposure.rescale_intensity(img,in_range=(np.quantile(img,0.03),1.5*np.quantile(img,0.99999)))
                    s_filename = f"Registered-R1_{s_marker}_{s_slide}-Scene-{s_scene}_c1_ORG.tif"
                elif (se_cyto == s_marker).sum() > 0:
                        cyto_scale = skimage.exposure.rescale_intensity(img,in_range=(np.quantile(img,0.03),1.5*np.quantile(img,0.99999)))
                        imgs_cyto.append(cyto_scale)
                if s_marker.find('dsDNA') == -1:
                    continue
                    s_filename = f"Registered-R{idx}_{s_marker}_{s_slide}-Scene-{s_scene}_c2_ORG.tif"
                    #hot pixels fix
                    blurred = median_filter(img, size=2)
                    difference = img.astype('int16') - blurred.astype('int16')
                    threshold = 10*np.std(difference)
                    i_filter = sum(img > int(threshold)).sum()/img.size
                    img[img > int(threshold)] = blurred[img > int(threshold)]
                    #save
                    io.imsave(f"{subdir}/{s_filename}", img,check_contrast=False)
            #dapis
            dapi = np.clip((dapi_scale),a_min=0,a_max=65535)
            dapi_gamma = skimage.exposure.adjust_gamma(dapi, gamma=0.6)
            #The mesmer job expects images of shape[(-1, 256, 256, 2)]. use cyto mip and dapi
            mip = np.stack(imgs_cyto).max(axis=0)
            cyto = np.clip((mip),a_min=0,a_max=65535)
            cyto_gamma = skimage.exposure.adjust_gamma(cyto, gamma=0.6)
            #de-noise        
            tv = denoise_tv_chambolle(dapi_gamma, weight=0.1)
            cyto_tv = denoise_tv_chambolle(cyto_gamma, weight=0.1)
            zdh = np.dstack((tv,cyto_tv,np.zeros(tv.shape)))
            io.imsave(f'{segdir}/Registered-R1_DNA1.Cyto_{s_slide}-Scene-{s_scene}_c1c2.png',(zdh*255).astype('uint8'))

        #break
'''

In [ ]:
#io.imsave('test.png',zdh[100:900,100:900])
#io.imshow(zdh[100:900,100:900])

In [ ]:
io.imshow(tv[100:600,100:600])

In [ ]:
io.imshow(cyto_tv[100:600,100:600])

In [ ]:
#io.imsave('test.png',zdh[100:900,100:900])

# segment MIBI data

In [ ]:
#export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/extras/CUPTI

In [ ]:
#export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/extras/CUPTI #enter in terminal
os.environ['LD_LIBRARY_PATH'] = '/usr/local/lib'
from skimage import io, measure, segmentation
from deepcell.applications import Mesmer
from deepcell.utils.plot_utils import create_rgb_image
import numpy as np
import pandas as pd

In [ ]:
df_file = pd.DataFrame(data=os.listdir(f'{segdir}/Mesmer_Images'),columns=['overlay_img'])
df_file['scene'] = [item.split('_')[2] for item in df_file.overlay_img]

df_seg = pd.DataFrame(data = os.listdir(f'{segdir}/Mesmer_Segmentation'),columns=['seg_image'])
df_seg['scene'] = [item.split(' ')[0] for item in df_seg.seg_image]

df_merge = df_file.merge(df_seg,on='scene',how='left')
ls_file = df_merge.loc[(df_merge.seg_image.isna()),'overlay_img'].tolist()


In [ ]:
ls_file #= ['Registered-R1_DNA1.Cyto_17-Scene-1_c1c2.png','Registered-R1_DNA1.Cyto_27-Scene-1_c1c2.png']

In [ ]:
%matplotlib inline
from deepcell.utils.plot_utils import make_outline_overlay

In [ ]:
#pretty example
image_mpp = 0.5
app = Mesmer()
for s_file in ls_file:
    img = io.imread(f'{segdir}/Mesmer_Images/{s_file}')
    img = img[90:300,590:800]
    s_scene = s_file.split('_')[2]
    a_test = np.expand_dims(img[:,:,0:2], axis=0)
    rgb_images = create_rgb_image(a_test, channel_colors=['blue', 'green'])
    segmentation_predictions = app.predict(a_test, image_mpp=image_mpp)
    overlay_data = make_outline_overlay(rgb_data=rgb_images, predictions=segmentation_predictions)
    idx=0
    fig, ax = plt.subplots(1, 2, figsize=(15, 15))
    ax[0].imshow(rgb_images[idx, ...])
    ax[1].imshow(overlay_data[idx, ...])
    ax[0].set_title('Raw data')
    ax[1].set_title('Predictions')
    plt.show()
    #cell_predictions = app.predict(a_test, image_mpp=image_mpp)
    segmentation_predictions_nuc = app.predict(a_test, image_mpp=image_mpp, compartment='nuclear')
    overlay_data_nuc = make_outline_overlay(rgb_data=rgb_images, predictions=segmentation_predictions_nuc)
    fig, ax = plt.subplots(1, 2, figsize=(15, 15))
    ax[0].imshow(rgb_images[idx, ...])
    ax[1].imshow(overlay_data_nuc[idx, ...])
    ax[0].set_title('Raw data')
    ax[1].set_title('Nuclear Predictions')
    plt.show()
    #labels = io.imread(f'{segdir}/Mesmer_Segmentation/{s_scene} nuclei{image_mpp} - Nuclei Segmentation Basins.tif')
    #cell_labels = io.imread(f'{segdir}/Mesmer_Segmentation/{s_scene} cell{image_mpp} - Cell Segmentation Basins.tif')
    break

In [ ]:
#loop
#'''DONE
#labels = io.imread(f'{s_scene} nuclei{nuc_diam} - Nuclei Segmentation Basins.tif')
#cell_labels = io.imread(f'{s_scene} cell{cell_diam} - Cell Segmentation Basins.tif')
image_mpp = 0.5
app = Mesmer()
for s_file in ls_file:
    img = io.imread(f'{segdir}/Mesmer_Images/{s_file}')
    s_scene = s_file.split('_')[2]
    print(f'processing {s_scene}')   
    a_test = np.expand_dims(img[:,:,0:2], axis=0)
    rgb_test = create_rgb_image(a_test, channel_colors=['blue', 'green'])
    #nuc
    test_predictions = app.predict(a_test, image_mpp=image_mpp, compartment='nuclear')
    labels = np.reshape(test_predictions[0],test_predictions[0].shape[0:2])
    io.imsave(f'{segdir}/Mesmer_Segmentation/{s_scene} nuclei{image_mpp} - Nuclei Segmentation Basins.tif',labels,check_contrast=False)
    #cell
    cell_predictions = app.predict(a_test, image_mpp=image_mpp)
    cell_labels = np.reshape(cell_predictions[0],cell_predictions[0].shape[0:2])
    #clean up and save cells
    df = pd.DataFrame(measure.regionprops_table(label_image=cell_labels,intensity_image=labels,properties=('label','area','mean_intensity')))
    ls_drop = df[df.mean_intensity==0].label
    cell_labels[np.isin(cell_labels, ls_drop)] = 0
    io.imsave(f'{segdir}/Mesmer_Segmentation/{s_scene} cell{image_mpp} - Cell Segmentation Basins.tif',cell_labels,check_contrast=False)
    print(f'done with {s_scene}')    
    #break
#'''

## match

In [ ]:
from numba import jit
import time
from skimage import io, measure, segmentation
#from deepcell.applications import Mesmer
#from deepcell.utils.plot_utils import create_rgb_image
import numpy as np
import pandas as pd
#oldcontainer,d_replace = segment.nuc_to_cell(labels,cell_labels)

@jit(nopython=True, parallel=True)
def _relabel_numba(ai_cell_labels, ai_cellid_relabel, o_dtype=np.uint32):
    '''
    version: 2021-12-00

    input:
        ai_cell_labels: (n,) one dimensional numpy array with cell labels.
        ai_cellid_relabel: (n,2) dimensional numpy array which maps original cell_id to nuc_id.
        o_dtype: data type inside the image array. standard is np.uint32,
            because 32bit tiffs can be opened with most tiff compatible software.

    output:
        ai_cell_relabeld: (n,) one dimensional numpy array with cell labels that match the nucleus.

    description:
        use numba to quickly iterate over each label and replace pixels with new pixel values.
    '''
    # handle input
    ai_cell_labels = ai_cell_labels.astype(o_dtype)
    ai_cellid_relabel = ai_cellid_relabel.astype(o_dtype)

    # do match
    ai_cell_relabeld = ai_cell_labels.copy()
    for i_cellid_cell, i_cellid_nuc in ai_cellid_relabel:
        ai_cell_relabeld[ai_cell_labels == i_cellid_cell] = i_cellid_nuc

    # output
    return(ai_cell_relabeld)

def _cell_to_nuc(ai_nuc_labels, ai_cell_labels):
    '''
    version: 2021-12-00

    input:
        ai_nuc_labels: (n,) one dimensional numpy array with nucleus labels.
        ai_cell_labels: (n,) one dimensional numpy array with cell labels.

    output:
        ai_cellid_relabel: (n,2) dimensional numpy array which maps original cell_id to nuc_id.

    description:
        associate the largest nucleus contained in each cell segmentation.
    '''
    # get cell dominant nuclei label relationship
    lli_cellid_relabel = []  # numba can't handle dictionary

    # iterate over each cell label
    for i_cellid in np.unique(ai_cell_labels):
        if i_cellid == 0:
            continue

        # find all non-zero nuclei label contained within that cell mask
        ai_nuc = ai_nuc_labels[ai_cell_labels == i_cellid]
        ai_nuc = ai_nuc[ai_nuc != 0]
        ai_nuc_unique = np.unique(ai_nuc)
        # multiple nuclei, choose largest (most common pixels, i.e. mode)
        if len(ai_nuc_unique) > 1:
            ai_value, ai_count = np.unique(ai_nuc, return_counts=True) # does not work with numba
            i_nucid_mode = ai_value[np.argmax(ai_count)]
            if (i_cellid != i_nucid_mode):
                lli_cellid_relabel.append([i_cellid, i_nucid_mode])
        # one nucleus
        elif len(ai_nuc_unique) == 1:
            i_nucid_one = ai_nuc[0]
            if (i_cellid != i_nucid_one):
                lli_cellid_relabel.append([i_cellid, i_nucid_one])
        # zero nuclei
        else:
            lli_cellid_relabel.append([i_cellid, 0])

    # output
    ai_cellid_relabel = np.array(lli_cellid_relabel) #, dtype=np.int64)
    return(ai_cellid_relabel)

def relabel_cells(labels,cell_labels):
    start = time.time()
    # ravel
    ti_nuc_shape = labels.shape
    ai_nuc_labels = labels.ravel()
    ti_cell_shape = cell_labels.shape
    ai_cell_labels = cell_labels.ravel()

    # match cell id with nuclei id
    print('get _cell_to_nuc_ listing ...')
    ai_cellid_relabel = _cell_to_nuc(ai_nuc_labels=ai_nuc_labels, ai_cell_labels=ai_cell_labels)
    
    # set minimalistic variable type
    print('running numba to relabel cell labels ...')
    if (ai_cellid_relabel.max() < 2**8):
        o_dtype = np.uint8
    elif (ai_cellid_relabel.max() < 2**16):
        o_dtype = np.uint16
    elif (ai_cellid_relabel.max() < 2**32):
        o_dtype = np.uint32
    else:
        # bue 20210810: currently, this will never happen. cellpose hardcoded the mask dtype to uint32.
        # this limits cell indexing to: 2**32 -1 = 4'294'967'295 [cell].
        o_dtype = np.uint64
    # numba re-label cell id
    ai_cell_relabeled = _relabel_numba(ai_cell_labels=ai_cell_labels, ai_cellid_relabel=ai_cellid_relabel, o_dtype=o_dtype)

    # unravel
    ai_nuc_labels.shape = ti_nuc_shape
    ai_cell_relabeled.shape = ti_cell_shape
    end = time.time()
    print(f'time elapsed: {end - start}[sec]')
    return(ai_cell_relabeled)

#features
def extract_feat(labels,intensity_image, properties=('centroid','mean_intensity','area','eccentricity')):
    ''' 
    given labels and intensity image, extract features to dataframe
    '''
    props = measure.regionprops_table(labels,intensity_image, properties=properties)
    df_prop = pd.DataFrame(props)
    return(df_prop)

def label_difference(labels,cell_labels):
    '''
    given matched nuclear and cell label IDs,return cell_labels minus labels
    '''
    overlap = cell_labels==labels
    ring_rep = cell_labels.copy()
    ring_rep[overlap] = 0
    return(ring_rep)

def index_format(df,s_scene,s_col='label'):
    df.index = [f'{s_scene.split("-Scene-")[0]}_scene{s_scene.split("-Scene-")[1].split("_")[0]}_cell{item}' for item in df.loc[:,s_col]]
    return(df)

def expand_label(labels,distance=3):
    '''
    expand the nucelar labels by a fixed number of pixels
    '''
    boundaries = segmentation.find_boundaries(labels,mode='outer') #thick
    shrunk_labels = labels.copy()
    shrunk_labels[boundaries] = 0
    background = shrunk_labels == 0
    distances, (i, j) = scipy.ndimage.distance_transform_edt(
                background, return_indices=True
            )

    grown_labels = labels.copy()
    mask = background & (distances <= distance)
    grown_labels[mask] = shrunk_labels[i[mask], j[mask]]
    ring_labels = grown_labels - shrunk_labels

    return(ring_labels, grown_labels) #shrunk_labels, grown_labels,

In [ ]:
#ls_scene_done = [] #['10-Scene-X1Y5']
#from mplex_image import segment, features
#ls_scene = sorted(set([item.split(' ')[0] for item in os.listdir(f'{segdir}/Mesmer_Segmentation') if not item.split('-Scene-')[1][1].isnumeric()])) #- set(ls_scene_done))
#len(ls_scene)

In [ ]:
ls_scene_done = [] #
ls_scene_done = sorted(set([item.split(' ')[0] for item in os.listdir(f'{segdir}/Mesmer_Segmentation') if item.find('matchedcell') > -1 if item.split('-Scene-')[1][0].isnumeric()]))#[] #['10-Scene-X1Y5']
#from mplex_image import segment, features
ls_scene = sorted(set([item.split(' ')[0] for item in os.listdir(f'{segdir}/Mesmer_Segmentation')  if item.split('-Scene-')[1][0].isnumeric()]) - set(ls_scene_done))
#ls_scene_skipped = ['274-Scene-X14Y1-46', '31-Scene-X16Y2-153']
len(ls_scene)

In [ ]:
ls_scene = ['17-Scene-1', '27-Scene-1']

In [ ]:
#matching #done
#'''DONE
image_mpp=0.5
for s_scene in ls_scene:
    print(s_scene)
    #s_scene = '46-Scene-X10Y7'
    labels = io.imread(f'{segdir}/Mesmer_Segmentation/{s_scene} nuclei{image_mpp} - Nuclei Segmentation Basins.tif')
    try:
        cell_labels = io.imread(f'{segdir}/Mesmer_Segmentation/{s_scene} cell{image_mpp} - Cell Segmentation Basins.tif')
    except:
        print(f'skip {s_scene}')
        continue
    cell_relabeled = relabel_cells(labels,cell_labels)
    io.imsave(f'{segdir}/Mesmer_Segmentation/{s_scene} matchedcell{image_mpp} - Cell Segmentation Basins.tif',cell_relabeled)
    #break
print(labels.max())
print(cell_labels.max())
print(cell_relabeled.max())
#'''

## extract features

In [ ]:
s_find = 'TMA'

In [ ]:
df_img.scene.nunique()

In [ ]:
#sorted(df_img.scene.unique()) #17, #27 

In [ ]:
#preprocess.dchange_fname({'HLA_Class_1':'HLA-Class-1'}) #,b_test=False

In [ ]:
os.chdir(f'{subdir}/{s_find}')
df_img = mpimage.parse_org()

In [ ]:
image_mpp=0.5
df_sample = pd.DataFrame()
for s_scene in df_img.scene.unique():
    labels = io.imread(f'{segdir}/Mesmer_Segmentation/{s_scene} nuclei{image_mpp} - Nuclei Segmentation Basins.tif')
    try:
        cell_labels = io.imread(f'{segdir}/Mesmer_Segmentation/{s_scene} matchedcell{image_mpp} - Cell Segmentation Basins.tif')
        print(f'extracting features {s_scene}')
    except:
        print(f'missing matched {s_scene}')
    s_dapi = f'Registered-R1_DNA1.Cyto_{s_scene}_c1c2.png'
    dapi = io.imread(f'{segdir}/Mesmer_Images/{s_dapi}')[:,:,0]
    #
    df_feat = extract_feat(labels,dapi, properties=(['label','area','eccentricity','centroid']))
    df_feat.columns = [f'{item}_nuclei' for item in df_feat.columns]
    df_feat = index_format(df_feat,s_scene,'label_nuclei')   
    cyto = label_difference(labels,cell_labels)
    d_loc = {'nuclei':labels,'cell':cell_labels,'cytoplasm':cyto}
    #markers (not duplicated
    df_scene = df_img[df_img.scene==s_scene].sort_values('round_ord')
    #df_scene = df_scene[(df_scene.marker!='RutheniumTetroxide') & (~df_scene.index.str.contains('Registered-R11_HistoneH3'))& (~df_scene.index.str.contains('Registered-R24_HistoneH3'))]
    #loop through markers
    for s_index in df_scene.index:
        df_marker = pd.DataFrame(index=df_feat.index)
        s_marker = df_scene.loc[s_index,'marker']
        #print(f'extracting features {s_marker}')
        if s_marker == 'DAPI':
            s_marker = s_marker + f'{df_scene.loc[s_index,"rounds"].split("R")[1]}'
        intensity_image = io.imread(s_index)
        for s_loc, a_loc in d_loc.items():
            df_marker_loc = extract_feat(a_loc,intensity_image, properties=(['mean_intensity','label']))
            df_marker_loc.columns = [f'{s_marker}_{s_loc}',f'label']
            df_marker_loc = index_format(df_marker_loc,s_scene) ##  df_marker_loc.index]
            df_marker_loc.drop(f'label',axis=1,inplace=True)
            df_marker = df_marker.merge(df_marker_loc, left_index=True,right_index=True,how='left',suffixes=('',f'{s_marker}_{s_loc}'))
        #extract nas
        df_na = df_marker[df_marker.isna().any(axis=1)]
        ls_na = [int(item.split('_cell')[1]) for item in df_na.index]
        ring_labels, grown_labels = expand_label(labels,distance=3)
        ring_labels[~np.isin(ring_labels,ls_na)] = 0
        grown_labels[~np.isin(grown_labels,ls_na)] = 0
        df_marker_ring = extract_feat(ring_labels,intensity_image, properties=(['mean_intensity','label']))
        df_marker_grown = extract_feat(grown_labels,intensity_image, properties=(['mean_intensity','label']))
        df_feat_na = df_marker_ring.merge(df_marker_grown,on='label',suffixes=('_ring','_grown'))
        df_feat_na = index_format(df_feat_na,s_scene)
        #fill in nas
        #break
        df_marker.loc[df_feat_na.index,f'{s_marker}_cytoplasm'] = df_feat_na.loc[:,f'mean_intensity_ring']
        df_marker.loc[df_feat_na.index,f'{s_marker}_cell'] = df_feat_na.loc[:,f'mean_intensity_grown']   
        df_feat = df_feat.merge(df_marker, left_index=True,right_index=True,how='left',suffixes=('',f'{s_marker}'))
        #break
    df_sample = df_sample.append(df_feat)
    #break
df_sample.to_csv(f'{segdir}/features_MIBI-{s_find}_MeanIntensity_Centroid_Shape.csv')
#df_sample.to_csv('features_IMC_MeanIntensity_Centroid_Shape.csv')

In [ ]:
print('done')

## Correlate to published

In [ ]:

s_find = 'TMA'
#s_find = 'ZTMA'
os.chdir(segdir)
df_mi = pd.read_csv((f'features_MIBI-{s_find}_MeanIntensity_Centroid_Shape.csv'),index_col=0)
#df_mi = df_mi.drop(['DNA2_cell', 'DNA2_nuclei', 'DNA2_cytoplasm'],axis=1).dropna()
#df_mi = df_mi.dropna() #618961 to 618811
#150 cells are na in the markers: probaly too small for the expansion (get borders erased them)
#df_mi.isna().sum().sort_values()[-3::].index
#df_mi[df_mi.drop(['DNA2_cell', 'DNA2_nuclei', 'DNA2_cytoplasm'],axis=1).isna().ArgonDimers_cytoplasm].sort_values(by='area_nuclei')
#df_mi.drop(['DNA2_cell', 'DNA2_nuclei', 'DNA2_cytoplasm'],axis=1).dropna()
#imc segmentation had 855668/ 844498 w/o liver cells
# mesmer seg has 618961 cells, 618811 NA free

In [ ]:
df_mi['slide_scene'] = [item.split('_')[0] for item in df_mi.index]
df_mi.head()

In [ ]:
#[item.split('_cell')[0].replace('scene','').split('-')[0] for item in df_mi.index]

In [ ]:
#drop na df_norm is the watershed 
df_norm = pd.read_csv(f'{codedir}/20211109_MIBI_FilteredMeanIntensity.csv',index_col=0,low_memory=False)
df_norm['slide_scene'] = [item.split('_')[0] for item in  df_norm.index]
#df_mi['slide_scene'] = [item.split('_cell')[0].replace('scene','') for item in df_mi.index]
#df_mi['slide_scene'] =[item.split('_cell')[0].replace('scene','').split('-')[0] for item in df_mi.index]
#df_norm = df_norm[~df_norm.slide_scene.isin(['Liver_X1Y1', 'Liver_X2Y1','31_X16Y2', '274_X14Y1'])]
#print(set(df_norm.slide_scene) - set(df_mi.slide_scene))
#print(set(df_mi.slide_scene) - set(df_norm.slide_scene))

In [ ]:
df_norm['slide_scene'] = [item.split('_')[0] for item in df_norm.index]
df_norm.head()

In [ ]:
%matplotlib inline
df_norm.slide_scene = df_norm.slide_scene.replace('t1','1')
fig, ax = plt.subplots(figsize=(4,5),dpi=200)
df_norm.groupby('slide_scene').SMA.count().sort_values().plot(kind='barh',ax=ax,width=1)
ax.yaxis.set_visible(False)
ax.set_xlim(0,7000)
ax.set_title('Original Segmentation')

In [ ]:
fig, ax = plt.subplots(figsize=(4,5),dpi=200)
ls_index = df_norm.groupby('slide_scene').SMA.count().sort_values().index
df_mi.groupby('slide_scene').area_nuclei.count().loc[ls_index].plot(kind='barh',ax=ax,width=1)
ax.yaxis.set_visible(False)
ax.set_xlim(0,7000)
ax.set_title('Mesmer Segmentation')

In [ ]:
#((df_ws - df_mes)/df_ws).sort_values(ascending=False)[0:8].index.tolist()

In [ ]:
 #[]
df_ws = df_norm.groupby('slide_scene').SMA.count().sort_values()
ls_index = df_norm.groupby('slide_scene').SMA.count().sort_values().index
df_mes = df_mi.groupby('slide_scene').area_nuclei.count().loc[ls_index]
ls_seg=  ((df_ws - df_mes)/df_ws).sort_values(ascending=False)[0:8].index.tolist()
ls_seg = []
df_both = pd.DataFrame(data=df_ws).rename({'SMA':'WS'},axis=1)
df_both['MES'] = df_mes
df_both.loc[ls_seg,'off'] = 'orange'
df_both.fillna('blue',inplace=True)
df_both.head()

In [ ]:
import seaborn as sns
from scipy import stats
fig, ax = plt.subplots(figsize=(3,3),dpi=300)
sns.regplot(x="WS", y="MES", data=df_both,ax=ax,line_kws={'color':'C0'},scatter_kws={'s':5,'alpha':0.8,
                                                             'color':list(df_both['off'].values),'label':['__','b'],
                                                            })
r, pval= stats.pearsonr(df_both['WS'], df_both['MES'])
ax.set_xlabel('Original cell count')
ax.set_ylabel('Mesmer cell count')
#ax.legend(['__', 'outliers'],labelcolor='blue')
ax.set_title(f'Original vs Mesmer\nPearson r={r:.2} p={pval:.2}')
ax.axis('equal')
ax.set_ylim(1500,8500)
ax.set_xlim(1500,8500)
print(pval)
plt.tight_layout()
fig.savefig(f'{qcdir}/scatterplot_Mesmer_vs_Original_cell_count.png')

In [ ]:
'''
ls_seg_two = []
df_both.loc[ls_seg_two,'two'] = 'orange'
df_both.fillna('blue',inplace=True)
fig, ax = plt.subplots(figsize=(4,3),dpi=300)
sns.regplot(x="WS", y="MES", data=df_both,ax=ax,
            scatter_kws={'s':5,'alpha':0.5,'color':list(df_both['two'].values),'label':['__','o']})
r, pval= stats.pearsonr(df_both.loc[df_both.two=='blue','WS'], df_both.loc[df_both.two=='blue','MES'])
ax.set_xlabel('Watershed cell count')
ax.legend(['__','two WS ROIS'])
ax.set_ylabel('Mesmer cell count')
ax.set_title(f'Watershed vs Mesmer\nPearson r={r:.2} p={pval:.2}')
print(pval)
plt.tight_layout()
fig.savefig(f'{qcdir}/scatterplot_Mesmer_vs_Wastershed_cell_count_two.png')
'''

In [ ]:
print(((df_ws - df_mes)/df_ws).mean())
print(((df_ws - df_mes)/df_ws).median())
print(len(df_ws))
#even greatest difference is 22%
ls_seg = ((df_ws - df_mes)/df_ws).sort_values()[-16::].index

In [ ]:
#ls_seg = set(ls_seg) - set(ls_seg_two)
ls_seg = [ '20', '19', '38', '16']
((df_ws - df_mes)/df_ws).sort_values()[-5::].index

In [ ]:
os.chdir(codedir)
os.chdir('Segmentation_Original')
df_ws = pd.DataFrame(index=sorted(os.listdir()))
df_ws.index = [item + '/segmentation_interior.png' for item in df_ws.index]

In [ ]:
os.chdir(codedir)
os.chdir('Segmentation/Mesmer_Segmentation')
df_mes_all = pd.DataFrame(index=sorted(os.listdir()))
df_mes = df_mes_all[df_mes_all.index.str.contains('matched')]

In [ ]:
df_seg = pd.DataFrame(index=os.listdir(f'{segdir}/Mesmer_Images'))
#df_seg.drop(['Mesmer_Segmentation','test'],inplace=True) #'features_IMC_MeanIntensity_Centroid_Shape.csv',
df_seg['slide'] = [item.split('-Scene-')[0].split('_')[-1] for item in df_seg.index]


In [ ]:
df_mes['slide'] = [item.split('-Scene-')[0] for item in df_mes.index]

In [ ]:
#BaselTMA_SP43_10.3kx13.7ky_6500x8000_6_20170928_29_19_X15Y2_216_a0_full_maks.tiff
df_ws['slide'] = [item.split('_Point')[1].split('/')[0] for item in df_ws.index]
ls_seg

In [ ]:
for s_seg in ls_seg:
    #s_seg = '104_X8Y4'
    s_file_mes = df_mes[df_mes.slide==s_seg].index[0]
    #print(s_seg)
    i_len = len(df_ws[df_ws.slide==s_seg].index)
    s_file_ws = df_ws[df_ws.slide==s_seg].index[0]
    s_file_img = df_seg[df_seg.slide==s_seg].index[0]
    matched = io.imread(f'{segdir}/Mesmer_Segmentation/{s_file_mes}')
    wshed = io.imread(f'{codedir}/Segmentation_Original/{s_file_ws}')
    img = io.imread(f'{segdir}/Mesmer_Images/{s_file_img}')
    #img = 
    fig, ax = plt.subplots(2,2,figsize=(5,4),dpi=200)
    ax=ax.ravel()
    img1 =io.imshow(matched,ax=ax[1],cmap='viridis',vmin=0,vmax=wshed.max())
    img2 = io.imshow(wshed,ax=ax[0],cmap='viridis',vmin=0,vmax=wshed.max())
    io.imshow(img,ax=ax[2])
    ax[0].set_title(f'watershed: {wshed.max()}')
    ax[1].set_title(f'Mesmer: {matched.max()}')
    ax[2].set_title('Image')
    ax[0].set_xlim(0,img.shape[1])
    ax[0].set_ylim(img.shape[0],0)
    ax[1].set_xlim(0,img.shape[1])
    ax[1].set_ylim(img.shape[0],0)
    fig.suptitle(s_seg,y=0.97)
    if s_seg == '137_X9Y3-39' or s_seg == '168_X10Y5-171':
        fig.colorbar(img2, ax=ax[0],fraction=.04)
    elif s_seg == '187_X3Y2-99' or s_seg == '167_X14Y8-211' or s_seg == '156_X15Y1-175':
        fig.colorbar(img1, ax=ax[1],fraction=.04)
    ax[3].remove()
    plt.tight_layout()
    fig.savefig(f'{qcdir}/segmentation_basins_{s_seg}.png')
    if i_len > 1:
        s_file_ws = df_ws[df_ws.slide_scene==s_seg].index[1]
        wshed = io.imread(f'{codedir}/ome/masks/{s_file_ws}')
        fig, ax = plt.subplots(figsize=(3,3),dpi=200)
        io.imshow(wshed,ax=ax,cmap='viridis')
    #break

### OLD testing seg

In [ ]:
    #test_predictions = app.predict(a_test, image_mpp=image_mpp, compartment='nuclear')
    #labels = np.reshape(test_predictions[0],test_predictions[0].shape[0:2])
    #io.imsave(f'{segdir}/Mesmer_Segmentation/{s_scene} nuclei{image_mpp} - Nuclei Segmentation Basins.tif',labels,check_contrast=False)
    #cell
    #cell_predictions = app.predict(a_test, image_mpp=image_mpp)
    #cell_labels = np.reshape(cell_predictions[0],cell_predictions[0].shape[0:2])

In [ ]:
io.imshow(labels)

In [ ]:
io.imshow(cell_labels)

In [ ]:
#cleaned up
io.imshow(cell_labels)

In [ ]:
df = pd.DataFrame(measure.regionprops_table(label_image=cell_labels,intensity_image=labels,properties=('label','area','mean_intensity')))
ls_drop = df[df.mean_intensity==0].label
cell_labels[np.isin(cell_labels, ls_drop)] = 0

In [ ]:
#a_test = np.expand_dims(img[:,:,0:2], axis=0)
#a_test.shape
#rgb_test = create_rgb_image(a_test, channel_colors=['blue', 'green'])
#whole cell
#app = Mesmer()
#test_predictions = app.predict(a_test, image_mpp=1)
#io.imshow(test_predictions[0])
#np.reshape(test_predictions[0],test_predictions[0].shape[0:2])
#io.imsave(f'{segdir}/test/predictions.tiff',np.reshape(test_predictions[0],test_predictions[0].shape[0:2]))

In [ ]:
'''
overlay_test = make_outline_overlay(rgb_data=rgb_test, predictions=test_predictions)
idx=0
i_size = 200
fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].imshow(rgb_test[idx, 0:i_size,0:i_size,:])
ax[1].imshow(overlay_test[idx, 0:i_size,0:i_size,:])
ax[0].set_title('Raw data')
ax[1].set_title('Predictions')
plt.show()
'''

In [ ]:
#nuclear
#test_predictions_nuc = app.predict(a_test, image_mpp=1, compartment='nuclear')
#io.imshow(test_predictions_nuc[0])
#np.reshape(test_predictions[0],test_predictions[0].shape[0:2])
#io.imsave(f'{segdir}/test/predictions_nuc.tiff',np.reshape(test_predictions_nuc[0],test_predictions_nuc[0].shape[0:2]))

In [ ]:
'''
overlay_test_nuc = make_outline_overlay(rgb_data=rgb_test, predictions=test_predictions_nuc)
idx=0
i_size = 200
fig, ax = plt.subplots(1, 2, figsize=(15, 15))
ax[0].imshow(rgb_test[idx, 0:i_size,0:i_size,:])
ax[1].imshow(overlay_test_nuc[idx, 0:i_size,0:i_size,:])
ax[0].set_title('Raw data')
ax[1].set_title('Predictions')
plt.show()
'''

In [ ]:
#napari clean up
'''
from skimage import measure
df = pd.DataFrame(measure.regionprops_table(label_image=cell_labels,intensity_image=nuc_labels,properties=('label','area','mean_intensity')))
ls_drop = df[df.mean_intensity==0].label
cell_labels[np.isin(cell_labels, ls_drop)] = 0
'''

# image processing

test differerent de-noising

In [ ]:
# also tried simple
# scipy.ndi.gaussian_filter().

In [ ]:
os.chdir(f'{segdir}/test')

In [ ]:
#a_scale = skimage.exposure.rescale_intensity(dapi,in_range=(np.quantile(dapi,0.03),1.5*np.quantile(dapi,0.9999)))
a_scale = dapi_scale

In [ ]:
io.imshow(a_scale[200:300,300:400])
plt.title('Clip .9999 quantile')

In [ ]:
io.imsave('scaled.png',a_scale[200:600,300:700])

In [ ]:
a_gamma = skimage.exposure.adjust_gamma(a_scale, gamma=0.4)

In [ ]:
io.imshow(a_gamma[200:300,300:400])
plt.title('Adjust gamma')

In [ ]:
io.imsave('gamma.png',a_gamma[200:600,300:700])

In [ ]:
a_gauss = scipy.ndimage.gaussian_filter(a_gamma,sigma=1)
io.imshow(a_gauss[200:300,300:400])
plt.title('Gaussian blurr')

In [ ]:
image = a_gamma[200:600,300:700]

In [ ]:
result_1 = unsharp_mask(image, radius=1, amount=1)
result_2 = unsharp_mask(image, radius=5, amount=2)
result_3 = unsharp_mask(image, radius=20, amount=1)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2,
                         sharex=True, sharey=True, figsize=(10, 10))
ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].set_title('Original image')
ax[1].imshow(result_1, cmap=plt.cm.gray)
ax[1].set_title('Enhanced image, radius=1, amount=1.0')
ax[2].imshow(result_2, cmap=plt.cm.gray)
ax[2].set_title('Enhanced image, radius=5, amount=2.0')
ax[3].imshow(result_3, cmap=plt.cm.gray)
ax[3].set_title('Enhanced image, radius=20, amount=1.0')

In [ ]:
io.imsave('result_1.png',(result_1*255).astype('uint8'))

In [ ]:
io.imsave('result_2.png',(result_2*255).astype('uint8'))

In [ ]:
io.imsave('result_3.png',(result_3*255).astype('uint8'))

In [ ]:
mip = np.stack(img_st).max(axis=0)

In [ ]:
#added = np.clip((img_st[0] + img_st[1]),a_min=0,a_max=65535)
#added_gamma = skimage.exposure.adjust_gamma(added, gamma=0.5)
#pretty good
noisy = a_gamma[200:600,300:700]
tv = denoise_tv_chambolle(noisy, weight=0.25)
#io.imsave('added_more_tv.png',(tv*255).astype('uint8'))

In [ ]:
#io.imshow(mip[200:600,300:700])

In [ ]:
#io.imsave('mip.png',mip[200:600,300:700])

In [ ]:
#mip_gamma = skimage.exposure.adjust_gamma(mip, gamma=0.5)

In [ ]:
#io.imshow(mip_gamma[200:600,300:700])

In [ ]:
#io.imsave('gmip.png',mip_gamma[200:600,300:700])

In [ ]:
#denoise
noisy = a_gamma[200:300,300:400]

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(8, 6), dpi=300,
                       sharex=True, sharey=True)
ax[0, 0].imshow(noisy,cmap='gray')
ax[0, 0].axis('off')
ax[0, 0].set_title('Noisy')
ax[0, 1].imshow(denoise_tv_chambolle(noisy, weight=0.1),cmap='gray')
ax[0, 1].axis('off')
ax[0, 1].set_title('TV (0.1)')
ax[0, 2].imshow(denoise_bilateral(noisy, sigma_color=0.05, sigma_spatial=15, 
                ),cmap='gray')
ax[0, 2].axis('off')
ax[0, 2].set_title('Bilateral (0.05)')
ax[1, 0].imshow(denoise_wavelet(noisy,  rescale_sigma=True),cmap='gray')
ax[1, 0].axis('off')
ax[1, 0].set_title('Wavelet denoising')

ax[1, 1].imshow(denoise_tv_chambolle(noisy, weight=0.2),cmap='gray')
ax[1, 1].axis('off')
ax[1, 1].set_title('TV (0.25)')
ax[1, 2].imshow(denoise_bilateral(noisy, sigma_color=.2,sigma_spatial=15, 
                ),cmap='gray')
ax[1, 2].axis('off')
ax[1, 2].set_title('Bilateral (0.2)')

fig.tight_layout()

In [ ]:
noisy = added[200:250,300:350]
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(8, 6), dpi=300,
                       sharex=True, sharey=True)
ax[0, 0].imshow(noisy)
ax[0, 0].axis('off')
ax[0, 0].set_title('Noisy')
ax[0, 1].imshow(denoise_tv_chambolle(noisy, weight=0.1))
ax[0, 1].axis('off')
ax[0, 1].set_title('TV')
ax[0, 2].imshow(denoise_bilateral(noisy, sigma_color=0.05, sigma_spatial=15,
                ))
ax[0, 2].axis('off')
ax[0, 2].set_title('Bilateral')
ax[1, 0].imshow(denoise_wavelet(noisy,  rescale_sigma=True))
ax[1, 0].axis('off')
ax[1, 0].set_title('Wavelet denoising')

ax[1, 1].imshow(denoise_tv_chambolle(noisy, weight=0.2))
ax[1, 1].axis('off')
ax[1, 1].set_title('(more) TV')
ax[1, 2].imshow(denoise_bilateral(noisy, sigma_color=0.1, sigma_spatial=15,
                ))
ax[1, 2].axis('off')
ax[1, 2].set_title('(more) Bilateral')

fig.tight_layout()

In [ ]:
added_gamma = skimage.exposure.adjust_gamma(added, gamma=0.5)
#pretty good
noisy = added_gamma[200:600,300:700]
tv = denoise_tv_chambolle(noisy, weight=0.25)
io.imsave('added_more_tv.png',(tv*255).astype('uint8'))

In [ ]:
noisy = added_gamma[200:250,300:350]
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(8, 6), dpi=300,
                       sharex=True, sharey=True)
ax[0, 0].imshow(noisy)
ax[0, 0].axis('off')
ax[0, 0].set_title('Noisy')
ax[0, 1].imshow(denoise_tv_chambolle(noisy, weight=0.1))
ax[0, 1].axis('off')
ax[0, 1].set_title('TV')
ax[0, 2].imshow(denoise_bilateral(noisy, sigma_color=0.05, sigma_spatial=15,
                ))
ax[0, 2].axis('off')
ax[0, 2].set_title('Bilateral')
ax[1, 0].imshow(denoise_wavelet(noisy,  rescale_sigma=True))
ax[1, 0].axis('off')
ax[1, 0].set_title('Wavelet denoising')

ax[1, 1].imshow(denoise_tv_chambolle(noisy, weight=0.25))
ax[1, 1].axis('off')
ax[1, 1].set_title('(more) TV')
ax[1, 2].imshow(denoise_bilateral(noisy, sigma_color=0.1, sigma_spatial=15,
                ))
ax[1, 2].axis('off')
ax[1, 2].set_title('(more) Bilateral')

fig.tight_layout()

In [ ]:
#pretty good
noisy = added_gamma[200:600,300:700]
tv = denoise_tv_chambolle(noisy, weight=0.25)
io.imsave('added_more_tv.png',(tv*255).astype('uint8'))

In [ ]:
io.imshow(noisy)

In [ ]:

footprint = morphology.disk(2)
res = morphology.white_tophat(noisy, footprint)
fig, ax = plt.subplots(ncols=3, figsize=(10, 5),dpi=300)
ax[0].set_title('Original')
ax[0].imshow(noisy, cmap='gray')
ax[1].set_title('White tophat')
ax[1].imshow(res, cmap='gray')
ax[2].set_title('Complementary')
ax[2].imshow(noisy - res, cmap='gray')

plt.show()

In [ ]:
tv = denoise_tv_chambolle(noisy, weight=0.25)
footprint = morphology.disk(2)
res = morphology.white_tophat(tv, footprint)
fig, ax = plt.subplots(ncols=3, figsize=(10, 5),dpi=300)
ax[0].set_title('Original')
ax[0].imshow(tv, cmap='gray')
ax[1].set_title('White tophat')
ax[1].imshow(res, cmap='gray')
ax[2].set_title('Complementary')
ax[2].imshow(tv - res, cmap='gray')

plt.show()

In [ ]:
noisy = added_gamma[200:300,300:400]
image = denoise_tv_chambolle(noisy, weight=0.25)
result_1 = unsharp_mask(image, radius=1, amount=1)
result_2 = unsharp_mask(image, radius=5, amount=1)
result_3 = unsharp_mask(image, radius=20, amount=1)

fig, axes = plt.subplots(nrows=2, ncols=2,
                         sharex=True, sharey=True, figsize=(10, 10))
ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].set_title('Original image')
ax[1].imshow(result_1, cmap=plt.cm.gray)
ax[1].set_title('Enhanced image, radius=1, amount=1.0')
ax[2].imshow(result_2, cmap=plt.cm.gray)
ax[2].set_title('Enhanced image, radius=5, amount=1.0')
ax[3].imshow(result_3, cmap=plt.cm.gray)
ax[3].set_title('Enhanced image, radius=20, amount=1.0')

In [ ]:
noisy = added_gamma[200:300,300:400]
image = denoise_tv_chambolle(noisy, weight=0.25)
result_1 = unsharp_mask(image, radius=1, amount=1)
result_2 = unsharp_mask(image, radius=1, amount=2)
result_3 = unsharp_mask(image, radius=1, amount=4)

fig, axes = plt.subplots(nrows=2, ncols=2,
                         sharex=True, sharey=True, figsize=(10, 10))
ax = axes.ravel()

ax[0].imshow(image, cmap=plt.cm.gray)
ax[0].set_title('Original image')
ax[1].imshow(result_1, cmap=plt.cm.gray)
ax[1].set_title('Enhanced image, radius=1, amount=1.0')
ax[2].imshow(result_2, cmap=plt.cm.gray)
ax[2].set_title('Enhanced image, radius=1, amount=2.0')
ax[3].imshow(result_3, cmap=plt.cm.gray)
ax[3].set_title('Enhanced image, radius=1, amount=4.0')

In [ ]:
#pretty good
noisy = added_gamma[200:600,300:700]
tv = denoise_tv_chambolle(noisy, weight=0.25)
unsharp = unsharp_mask(tv, radius=1, amount=4)
io.imsave('added_more_tv_unsharp.png',(unsharp*255).astype('uint8'))

In [ ]:
#pretty bad

tvu = denoise_tv_chambolle(unsharp, weight=0.1)
io.imsave('added_more_tv_us_tv.png',(tvu*255).astype('uint8'))